In [1]:
# Explanatory notebook to convert this database with specific (radical) repetition units and end groups to the PSMILES of the Ramprasad-Group
import os
import pandas as pd
import re
import warnings

In [2]:
RAW_CSV_PATH = os.path.join("../" + "cloud_point" + "./cloud_points_data.csv")
# RAW_CSV_PATH = "./glass_transition_temperature_data.csv"

raw_df = pd.read_csv(RAW_CSV_PATH, sep=";", decimal=",")
raw_df

,reference,polymer_type,polymer_type_style,polymer_architecture,polymerisation_type,SMILES_start_group,SMILES_end_group,SMILES_repeating_unitA,molpercent_repeating_unitA,SMILES_repeating_unitB,...,additive2_concentration_molar,additive2_concentration_weight_percent,cloud_point,N/A,def_type,pH,identifier,comment,tacticity,rating
0,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[C](C)(C)(C#N),[C](C)(C)C#N,[CH2][CH](C(=O)NC(C)C),0.8,[CH2][CH](C(=O)O),...,NaN,NaN,32.1,NaN,A,NaN,PNIPAM-co-AA,NaN,NaN,NaN
1,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[C](C)(C)(C#N),[C](C)(C)C#N,[CH2][CH](C(=O)NC(C)C),0.8,[CH2][CH](C(=O)O),...,NaN,NaN,43.0,NaN,A,NaN,PNIPAM-cycloprop-7.5,NaN,NaN,NaN
2,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[C](C)(C)(C#N),[C](C)(C)C#N,[CH2][CH](C(=O)NC(C)C),0.8,[CH2][CH](C(=O)O),...,NaN,NaN,44.0,NaN,A,NaN,PNIPAM-cyclobut-7.5,NaN,NaN,NaN
3,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[C](C)(C)(C#N),[C](C)(C)C#N,[CH2][CH](C(=O)NC(C)C),0.8,[CH2][CH](C(=O)O),...,NaN,NaN,45.5,NaN,A,NaN,PNIPAM-cyclopent-7.5,NaN,NaN,NaN
4,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[C](C)(C)(C#N),[C](C)(C)C#N,[CH2][CH](C(=O)NC(C)C),0.8,[CH2][CH](C(=O)O),...,NaN,NaN,43.0,NaN,A,NaN,PNIPAM-cyclohept-7.5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,10.1021/ma1005759,homo,NaN,graft,RAFT,[C](C)(C#N)(CCC(=O)NCC(F)(F)C(F)(F)C(F)(F)C(F)...,[CH2]CCCCCCCCCCCCCCC,[CH2][C](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,51.7,NaN,C,NaN,P15,632 nm,NaN,NaN
907,10.1021/ma1005759,homo,NaN,graft,RAFT,[C](C)(C#N)(CCC(=O)NCC(F)(F)C(F)(F)C(F)(F)C(F)...,[S](SCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)...,[CH2][C](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,50.3,NaN,C,NaN,P16,632 nm,NaN,NaN
908,10.1021/ma1005759,homo,NaN,graft,RAFT,[C](C)(C#N)(CCC(=O)N(CCCCCCCCCCCCCCCCCC)CCCCCC...,[S]SC,[CH2][C](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,48.9,NaN,C,NaN,P17,632 nm,NaN,NaN
909,10.1021/ma1005759,homo,NaN,graft,RAFT,[C](C)(C#N)(CCC(=O)N(CCCCCCCCCCCCCCCCCC)CCCCCC...,[S](SCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)...,[CH2][C](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,48.9,NaN,C,NaN,P18,632 nm,NaN,NaN


In [3]:
# transform every monomers  "[atoms and numbers]" string-part to "*atoms and numbers" e.g.:
# [CH2][CH](C(=O)NC1CCCCCC1) -> [*]CC[*](C(=O)NC1CCCCCC1)
# therefore all "[content]" must be searched then
    # the first must be replaced with "[*]content-H and following numbers"
    # the second must be replaced with "content-h and following numbers[*]"
def convert_explicit_SMILES_to_PSMILES(explicit_SMILES: str, chain_pos: str = "monomer"):
    all_explicits = re.findall(r"\[[^\]\*]*\]", explicit_SMILES)
    replacements = [radical.replace("[", "").replace("]", "") for radical in all_explicits]
    replacements = [re.sub("H[0-9]?", "", radical) for radical in replacements]
    if len(all_explicits) == 0:
        return explicit_SMILES
    PSMILES = explicit_SMILES
    match chain_pos:
        case "monomer":
            replacements[0] = "[*]" + replacements[0]
            replacements[1] = replacements[1] + "[*]"
            PSMILES = PSMILES.replace(all_explicits[0], replacements[0], 1)
            PSMILES = PSMILES.replace(all_explicits[1], replacements[1], 1)
            if len(all_explicits) != 2:
                warnings.warn("\n" + explicit_SMILES + ": " + "Too many OR little explicits found. Is there a radical in the monomer, or is this an end group?" +
                              " Is " + "\n" + PSMILES + " the right replacement?")
        case "end":
            replacements[0] = "[*]" + replacements[0]
            PSMILES = PSMILES.replace(all_explicits[0], replacements[0], 1)
            if len(all_explicits) != 1:
                warnings.warn("\n" + explicit_SMILES + ": " + "None or too many explicits found. Is there a radical in the end group, or is it a monomer?" +
                              " Is " + "\n" + PSMILES + " the right replacement?")

    return PSMILES
convert_explicit_SMILES_to_PSMILES("[CH2][C](C)(C(=O)OCC[N+](C)(C)CCCS(=O)(=O)[O-]","monomer")

C:\Users\Nex\AppData\Local\Temp\ipykernel_15544\2758066532.py:20: UserWarning: 
[CH2][C](C)(C(=O)OCC[N+](C)(C)CCCS(=O)(=O)[O-]: Too many OR little explicits found. Is there a radical in the monomer, or is this an end group? Is 
[*]CC[*](C)(C(=O)OCC[N+](C)(C)CCCS(=O)(=O)[O-] the right replacement?
  warnings.warn("\n" + explicit_SMILES + ": " + "Too many OR little explicits found. Is there a radical in the monomer, or is this an end group?" +


'[*]CC[*](C)(C(=O)OCC[N+](C)(C)CCCS(=O)(=O)[O-]'

In [4]:
# get all headers with SMILES and convert them to PSMILES, also split between SMILES + repeating and none
end_group_headers = [header for header in raw_df.columns if "end" and "group" in header]
monomer_headers = [header for header in raw_df.columns if "SMILES_repeating" in header]
print(end_group_headers, monomer_headers)
for end_group in end_group_headers:
    raw_df[end_group] = raw_df[end_group].apply(lambda x: convert_explicit_SMILES_to_PSMILES(x, "end") if type(x) == str else x)
for monomer in monomer_headers:
    raw_df[monomer] = raw_df[monomer].apply(lambda x: convert_explicit_SMILES_to_PSMILES(x, "monomer") if type(x) == str else x)
raw_df

['SMILES_start_group', 'SMILES_end_group'] ['SMILES_repeating_unitA', 'SMILES_repeating_unitB', 'SMILES_repeating_unitC', 'SMILES_repeating_unitD', 'SMILES_repeating_unitE']


C:\Users\Nex\AppData\Local\Temp\ipykernel_15544\2758066532.py:26: UserWarning: 
[C](C)(C#N)(CCC(=O)NCC[N+](C)(C)C): None or too many explicits found. Is there a radical in the end group, or is it a monomer? Is 
[*]C(C)(C#N)(CCC(=O)NCC[N+](C)(C)C) the right replacement?
  warnings.warn("\n" + explicit_SMILES + ": " + "None or too many explicits found. Is there a radical in the end group, or is it a monomer?" +
C:\Users\Nex\AppData\Local\Temp\ipykernel_15544\2758066532.py:26: UserWarning: 
[CH2]C=C(O[Si](C)(C)C)N(CC)CC: None or too many explicits found. Is there a radical in the end group, or is it a monomer? Is 
[*]CC=C(O[Si](C)(C)C)N(CC)CC the right replacement?
  warnings.warn("\n" + explicit_SMILES + ": " + "None or too many explicits found. Is there a radical in the end group, or is it a monomer?" +
C:\Users\Nex\AppData\Local\Temp\ipykernel_15544\2758066532.py:20: UserWarning: 
[CH2][C](C)(C(=O)N[C@H](CC)CO): Too many OR little explicits found. Is there a radical in the monomer, or 

,reference,polymer_type,polymer_type_style,polymer_architecture,polymerisation_type,SMILES_start_group,SMILES_end_group,SMILES_repeating_unitA,molpercent_repeating_unitA,SMILES_repeating_unitB,...,additive2_concentration_molar,additive2_concentration_weight_percent,cloud_point,N/A,def_type,pH,identifier,comment,tacticity,rating
0,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[*]C(C)(C)(C#N),[*]C(C)(C)C#N,[*]CC[*](C(=O)NC(C)C),0.8,[*]CC[*](C(=O)O),...,NaN,NaN,32.1,NaN,A,NaN,PNIPAM-co-AA,NaN,NaN,NaN
1,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[*]C(C)(C)(C#N),[*]C(C)(C)C#N,[*]CC[*](C(=O)NC(C)C),0.8,[*]CC[*](C(=O)O),...,NaN,NaN,43.0,NaN,A,NaN,PNIPAM-cycloprop-7.5,NaN,NaN,NaN
2,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[*]C(C)(C)(C#N),[*]C(C)(C)C#N,[*]CC[*](C(=O)NC(C)C),0.8,[*]CC[*](C(=O)O),...,NaN,NaN,44.0,NaN,A,NaN,PNIPAM-cyclobut-7.5,NaN,NaN,NaN
3,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[*]C(C)(C)(C#N),[*]C(C)(C)C#N,[*]CC[*](C(=O)NC(C)C),0.8,[*]CC[*](C(=O)O),...,NaN,NaN,45.5,NaN,A,NaN,PNIPAM-cyclopent-7.5,NaN,NaN,NaN
4,10.1016/j.fuel.2016.10.075,random,NaN,linear,FRP,[*]C(C)(C)(C#N),[*]C(C)(C)C#N,[*]CC[*](C(=O)NC(C)C),0.8,[*]CC[*](C(=O)O),...,NaN,NaN,43.0,NaN,A,NaN,PNIPAM-cyclohept-7.5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,10.1021/ma1005759,homo,NaN,graft,RAFT,[*]C(C)(C#N)(CCC(=O)NCC(F)(F)C(F)(F)C(F)(F)C(F...,[*]CCCCCCCCCCCCCCCC,[*]CC[*](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,51.7,NaN,C,NaN,P15,632 nm,NaN,NaN
907,10.1021/ma1005759,homo,NaN,graft,RAFT,[*]C(C)(C#N)(CCC(=O)NCC(F)(F)C(F)(F)C(F)(F)C(F...,[*]S(SCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,[*]CC[*](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,50.3,NaN,C,NaN,P16,632 nm,NaN,NaN
908,10.1021/ma1005759,homo,NaN,graft,RAFT,[*]C(C)(C#N)(CCC(=O)N(CCCCCCCCCCCCCCCCCC)CCCCC...,[*]SSC,[*]CC[*](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,48.9,NaN,C,NaN,P17,632 nm,NaN,NaN
909,10.1021/ma1005759,homo,NaN,graft,RAFT,[*]C(C)(C#N)(CCC(=O)N(CCCCCCCCCCCCCCCCCC)CCCCC...,[*]S(SCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,[*]CC[*](C)(C(=O)OCCOCCOCCOCCOC),1.0,NaN,...,NaN,NaN,48.9,NaN,C,NaN,P18,632 nm,NaN,NaN


In [5]:
# export converted table
pd.DataFrame.to_csv(raw_df, "./PSMILES_converted_cloud_point_data.csv", sep=";", decimal=",", index=False)
# pd.DataFrame.to_csv(raw_df, "./PSMILES_converted_tg_temp_data.csv", sep=";", decimal=",", index=False)

